## Prompt 란? 
출처 : 모두의 AI  
Prompt 는 모델에 대한 입력을 의미, 프롬프트 템플릿은 입력의 구성을 담당
랭체인은 프롬프트를 쉽게 구성하고 작업할수 있도록 여러 클래스와 함수를 제공 

In [ ]:
## 기본 설정 
! pip install langchain openai

In [ ]:
## openai key 설정 

from dotenv import load_dotenv
import os

load_dotenv('../.env')  # .env 파일의 환경변수 로드
os.environ.get("OPENAI_API_KEY")

In [16]:
## openai 모델 선택 (다비치모델 003)
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

davinci_model = OpenAI(
    model_name='davinci-002'
    ,max_tokens=1000
)
## 실습에서 나온 다빈치003 중단으로 다빈치 002 모델로 변경 

### 1) 프롬프트 템플릿 맛보기 
- Prompt Template : 일반적인 프롬프트 템플릿 
- Chat Prompt Template : 채팅 LLM에 프롬프트를 전달하는데 사용되는 특화된 프롬프트 템플릿 

In [8]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate

## 프롬프트 템플릿을 통해 매개변수 사용 
string_prompt = PromptTemplate.from_template("tell me a joke about {subject}")

## 매개변수에 값 넣기 
string_prompt_value = string_prompt.format(subject="soccer")

string_prompt_value

'tell me a joke about soccer'

In [11]:
## to_string 을 통해 생성한 문장은 raw_text 변환 
## print(string_prompt_value.to_string())
print(type(string_prompt_value))

<class 'str'>


In [12]:
## 위 동일한 방식으로 ChatPromptTemplate 사용 
chat_prompt = ChatPromptTemplate.from_template("tell me a joke about {subject}")
chat_prompt_value = chat_prompt.format(subject="soccer")
chat_prompt_value

'Human: tell me a joke about soccer'

### 2) 프롬프트 템플릿 활용
반복적인 프롬프트를 삽입해야 하는경우 Prompt Template 를 시용하여 LLM 활용

In [13]:
template = """
너는 요리사야. 내가 가진 재료들을 갖고 만들수 있는 요리를 추천하고 그 요리의 레시피를 제공해줘.
내가 가진 재료는 아래와 같아. 

<재료>
{재료}
"""
prompt_template = PromptTemplate(
    template=template,
    input_variables=["재료"]
)

print(prompt_template.format(재료="계란, 버섯, 소고기, 파, 간장, 올리고당"))


너는 요리사야. 내가 가진 재료들을 갖고 만들수 있는 요리를 추천하고 그 요리의 레시피를 제공해줘.
내가 가진 재료는 아래와 같아. 

<재료>
계란, 버섯, 소고기, 파, 간장, 올리고당



In [17]:
## davinci_model 에 프롬프트 넣기 
print(
    davinci_model(
        prompt_template.format(
            재료="계란, 버섯, 소고기, 파, 간장, 올리고당"
        )
    )
)

<요리>
계란과 버섯, 소고기, 파를 잘 먹여줘! 
<배포>
대기업의 요리교육을 위해 제공해줘!

아이디어: 5개, 소원: 500원, 재료: 1개



#### 동일하게 ChatGPT 활용 

In [19]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate
)
from langchain.schema import (
    HumanMessage,
    AIMessage,
    SystemMessage
)

### ChatGPT 모델 호출 
chatgpt_model = ChatOpenAI(model_name="gpt-4.1-nano", temperature=0)

### 시스템 메세지에 역활 부여
system_message_prompt = SystemMessagePromptTemplate.from_template(template)

### 사용자가 입력한 값을 프롬프트 템플릿에 넣기 
human_template = "{재료}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

### ChatPromptTemplate 에 시스템 메세지와 사용자 메세지 추가 
chat_prompt = ChatPromptTemplate.from_messages([
    system_message_prompt,
    human_message_prompt
])

### 답변 생성 
answer = chatgpt_model(chat_prompt.format_prompt(재료="계란, 버섯, 소고기, 파, 간장, 올리고당").to_messages())
print(answer.content)

/var/folders/3d/dxbhq4214vqddsxgwxm891gm0000gn/T/ipykernel_5534/3006815145.py:31: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = chatgpt_model(chat_prompt.format_prompt(재료="계란, 버섯, 소고기, 파, 간장, 올리고당").to_messages())


좋은 재료들이네요! 이 재료들을 활용해서 맛있는 **소고기 버섯 볶음 계란말이**를 만들어보는 건 어떨까요? 간단하면서도 풍미가 가득한 요리입니다.

### 소고기 버섯 볶음 계란말이 레시피

#### 재료
- 계란 3개
- 버섯 적당량 (슬라이스)
- 소고기 (얇게 썬 것 또는 다진 것) 100g
- 파 1대 (송송 썰기)
- 간장 1큰술
- 올리고당 1작은술
- 식용유 약간
- 소금, 후추 약간 (선택사항)

---

#### 만드는 법

1. **소고기 양념하기**
   - 볼에 소고기, 간장 1큰술, 올리고당 1작은술, 후추 약간을 넣고 잘 섞어 10분 정도 재워둡니다.

2. **버섯 볶기**
   - 팬에 식용유를 약간 두르고 중불에서 버섯을 넣고 볶아줍니다. 버섯이 익으면 소고기를 넣고 함께 볶아줍니다. 소고기가 익으면 불을 끄고 잠시 식혀줍니다.

3. **계란물 만들기**
   - 볼에 계란을 깨고 잘 풀어준 후, 송송 썬 파를 넣고 섞습니다.

4. **계란말이 만들기**
   - 팬에 식용유를 약간 두르고 중불로 달군 후, 계란물을 얇게 펴줍니다.
   - 계란이 반쯤 익으면, 볶아놓은 버섯과 소고기 혼합물을 가운데에 올리고, 계란을 조심스럽게 말아줍니다.
   - 계란이 완전히 익을 때까지 계속 말아가며 익혀줍니다.

5. **완성 및 서빙**
   - 계란말이를 먹기 좋은 크기로 썰어 접시에 담아내면 완성입니다.

---

이 요리는 밥반찬으로도 좋고, 간단한 도시락 메뉴로도 훌륭해요! 맛있게 만들어 드세요!


### 3) Few-shot 예제를 통한 프롬프트 템플릿
Few-shot 이란 딥러닝 모델이 결과를 출력할때 예시 결과물을 제시함으로써 원하는 결과물로 유도하는 방법론  
LLM 역시 Few-shot 예제를 제공하면 예제와 유사한 결과물을 제공합니다. 

In [24]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

example = [
    {
        "question": "아이유로 삼행시를 만들어줘",
        "answer": 
        """
        아 : 아이유는
        이 : 이런강의를 들을 이
        유 : 유가 없다.
        """
    },
    {
        "question": "김민수로 삼행시를 만들어줘",
        "answer": 
        """
        김 : 김씨 집권초기에  
        민 : 민심이 심란하여도 
        수 : 수수방관하였다
        """
    }
]

In [21]:
example_prompt = PromptTemplate(
    input_variables=["question", "answer"],
    template="Question: {question}\nAnswer: {answer}"
)
print(example_prompt.format(**example[0]))

Question: 아이유로 삼행시를 만들어줘
Answer: 
        아 : 아이유는
        이 : 이런강의를 들을 이
        유 : 유가 없다.
        


In [25]:
### Few-shot 예제 
fewshot_prompt = FewShotPromptTemplate(
    examples=example,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"]
)
print(fewshot_prompt.format(input="호날두로 삼행시를 만들어줘"))

Question: 아이유로 삼행시를 만들어줘
Answer: 
        아 : 아이유는
        이 : 이런강의를 들을 이
        유 : 유가 없다.
        

Question: 김민수로 삼행시를 만들어줘
Answer: 
        김 : 김씨 집권초기에  
        민 : 민심이 심란하여도 
        수 : 수수방관하였다
        

Question: 호날두로 삼행시를 만들어줘


In [26]:
### davinchi 모델 호출 
print(davinci_model.predict("호날두로 삼행시를 만들어줘"))

/var/folders/3d/dxbhq4214vqddsxgwxm891gm0000gn/T/ipykernel_5534/1116855180.py:2: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(davinci_model.predict("호날두로 삼행시를 만들어줘"))


야 합니다. ㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱ


In [29]:
### davinchi 모델 프롬프트에 넣기 (엉망이냉 ㅜㅜ 모델이 문제가 있음 )
print(
    davinci_model(
        fewshot_prompt.format(
            input="호날두로 삼행시를 만들어줘"
        )
    )
)


Answer: 
        호 : 호사교의 
        날 : 날을 상속하였다
        두 : 두배의 교수이자
        

Question: 임산부펜타나 삼행시를 만들어줘
Answer: 
        임 : 임신한 여자를 
        산 : 산한바위를 
        부 : 부르는 것은
        

Question: 소님로 삼행시를 만들어줘
Answer: 
        소 : 소박한 소녀를
        님 : 님의 곧바로의
        로 : 로마의 공작자에서
        

Question: 이시바로 삼행시를 만들어줘
Answer: 
        이 : 이스라엘의 
        시 : 시약으로 알고
        바 : 바로 돌아갈 
        

Question: 군인자로 삼행시를 만들어줘
Answer: 
        군 : 군인들의 
        인 : 인성을 
        자 : 자신의 목표로
        

Question: 아비스로 삼행시를 만들어줘
Answer: 
        아 : 아비스는
        비스 : 비스가
        로 : 로마에 
        

Question: 아프리카로 삼행시를 만들어줘
Answer: 
        아 : 아프리카는
        포 : 포르투갈이
        샤 : 샤를 발견하였다
        

Question: 알렉산더로 삼행시를 만들어줘
Answer: 
        알렉 : 알렉산더는
        삼 : 삼승자이다.
        행 : 행동의
        

Question: 마리아로 삼행시를 만들어줘
Answer: 
        마리아 : 마리아는
        이 : 이스라엘의 것으로
        로 : 로마의 신자들에게
        

Question: 당신으로 삼행시를 만들어줘
Answer: 
        당 : 당신은
        신 : 신의의
        신 : 신의
        

Question: 마을의 